In [2]:
# Each image is a 28x28 (784) pixel, so we can treat it as 28x28 matrix
# with input values from 0 to 255
# Approach for neural networks is to "flatten" each image into 
# a vector of 784x1 with each entry as an input so we will have 784 inputs

# we will be having two hidden layers and 10 outputs as we will be having
# number from 0 to 9 and be shown as a one hot vector 
# 0 - 1 0 0 0 0 0 0 0 0 0
# 5 - 0 0 0 0 1 0 0 0 0 0

# We will be using softmax activation function for the output layer
# because it will be a multinomial probability

# 1. Prepare the data and preprocess. Divide datasets to training, validation and test
# 2. Outline the model and choose activation functions
# 3. Set appropriate advanced optimizers and the loss function
# 4. Make it learn
# 5. Test the accuracy of the model

In [4]:
# Import the packages
import numpy as np
import tensorflow as tf

# Tensorflow dataset provider
import tensorflow_datasets as tfds

In [11]:
# Load the data
# default path is c:/users/uname/tensorflow_datasets
# as_supervised=True loads the data in a 2-tuple struct[input,target]
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

# Extract the training set and testing set
# mnist already has train and test
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

# However, we split somefor our validation from train
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples # from with_info
num_validation_samples = tf.cast(num_validation_samples, tf.int64) # Cast to int

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64) 

# Scale our data to make it numerically stable
# to make inputs between 0 and 1
def scale(image, label):
    image = tf.cast(image,tf.float32)
    image /= 255. # divide by 255 as each pixel will max of 255 and add (.) to indicate that we want a float as output
    return image, label

# data.map(function) applies transformation to a dataset
# just remember that the custom function should have (input, label)
# as arguments
scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)


# Normal practice is shuffling and then batching

# Set buffer size as we can shuffle the data at once, dataset too big, like a batch
BUFFER_SIZE = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

# Extract validation and test samples 
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)


# Batch size. 
# Apply on batch size on training 
# Not applied on validation data as we only forward propagate the validation
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)

# Since we find the model expect the validation set in batch form too
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

# Validation data must have the same shape as train and test
# mnist data is iterable and in 2-tuple format due to as_supervised=True
# divided to input and target or (image,label) 
validation_inputs, validation_targets = next(iter(validation_data)) #Iter goes to iteration and next will load the iter


In [17]:
# Outline the model
# 784 Inputs -> Hidden Layer (50 nodes) -> Hidden Layer (50n) -> Output 10
# Width and Depth

# Hyperparameters
input_size = 784
output_size = 10
hidden_layer_size = 100 # assumed all hidden layer is same size

model = tf.keras.Sequential([
    # Input layer, each data is 28x28x1, we have to flatten because we dont use CNN, preprocess argument
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    # Builds the layers dense(out_size_of_first, activation, )
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax'),
])

In [18]:
# Choose optimizer and loss function

# For loss
# Binary_crossentropy is for binary data
# categorical is for one hot encoded targets
# sparse categorical does the one hot encoding
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [19]:
# Training the model
NUM_EPOCHS = 5

model.fit(train_data, epochs= NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 3s - loss: 0.3225 - accuracy: 0.9095 - val_loss: 0.1806 - val_accuracy: 0.9502
Epoch 2/5
540/540 - 2s - loss: 0.1348 - accuracy: 0.9602 - val_loss: 0.1309 - val_accuracy: 0.9630
Epoch 3/5
540/540 - 2s - loss: 0.0974 - accuracy: 0.9711 - val_loss: 0.0936 - val_accuracy: 0.9708
Epoch 4/5
540/540 - 2s - loss: 0.0762 - accuracy: 0.9762 - val_loss: 0.0754 - val_accuracy: 0.9778
Epoch 5/5
540/540 - 2s - loss: 0.0608 - accuracy: 0.9807 - val_loss: 0.0607 - val_accuracy: 0.9815


In [21]:
# Epoch
# Batch Size
# Time it finished
# Training_loss should almost be decreasing
# accuracy - what % is output is equal to target (among batches) should be same trend as training loss
# val_loss = 
# val_acc = true accuracy for the epoch, check last (96.72%) increased hidden layer size to 100 then acc goes to 98.15%


# Models should have val_accuracy of at least 97%

In [24]:
# Testing the model
test_loss, test_acc = model.evaluate(test_data)
print('Test loss: {0:.2f}. Test Accuracy: {1:.2f}%'.format(test_loss,test_acc*100))

# Test accuracy should be close to validation accuracy

1/1 [==============================] - 1s 680ms/step - loss: 0.0887 - accuracy: 0.9725
Test loss: 0.09. Test Accuracy: 97.25%
